## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o3/o3_bincore6.csv", index_col=0)
print(bin6_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_3.reset_index(inplace=True, drop=True)

print('shape -', bin6_3.shape)
print('reset_index 완료')
print('input data shape')
bin6_3.head()

(33250757, 2)
shape - (33250757, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_3['label'].value_counts())

256
0    33215560
1       35197
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_3[bin6_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_3', len(ls_idx_bin))

# loc 로 수정필요
bin6_3_Ngram = bin6_3.loc[ls_idx_bin,:].copy()

563152
나머지 0
최종 길이 563152
bin6_3 563152


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

35197.0
0 5644511
1000 10690981
2000 27345652
3000 19032419
4000 25985660
5000 3340000
6000 21774108
7000 24724382
8000 12037836
9000 16336515
10000 29774734
11000 30737432
12000 7367590
13000 14451571
14000 24122628
15000 24555934
16000 27411194
17000 32722118
18000 7085239
19000 23983717
20000 30267951
21000 2144240
22000 21104113
23000 11328642
24000 15517384
25000 32847903
26000 25270968
27000 9170872
28000 23512499
29000 18614260
30000 7013604
31000 22942211
32000 9573676
33000 28621216
34000 7798298
35000 7546364
완료
35197


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_3['label'].value_counts()[1])

35197
35197


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_3_Ngram])
final.shape

(1126304, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_3_onehot_Ngram = pd.concat([final['label'], bc6_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_3_onehot_Ngram.shape)

원핫인코딩완료
(1126304, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_3 = bc6_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_3 = bc6_3_onehot_Ngram['label'].to_numpy()
print(x_bc6_3.shape, x_bc6_3.shape)

x_bc6_3 = x_bc6_3.reshape(-1, right_idx, x_bc6_3.shape[1])
y_bc6_3 = y_bc6_3.reshape(-1, right_idx, 1)

print(x_bc6_3.shape, y_bc6_3.shape)

(1126304, 256) (1126304, 256)
(70394, 16, 256) (70394, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_3.shape[0])

x_bc6_3 = x_bc6_3[p]
y_bc6_3 = y_bc6_3[p]

print(x_bc6_3.shape, y_bc6_3.shape)

(70394, 16, 256) (70394, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [11]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

print('======Training stage======')
model1.fit(x_bc6_3,
           y_bc6_3,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

model1.save('gcc6_bin_core_s16_h24_o3.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 49275 samples, validate on 21119 samples
Epoch 1/500
49275/49275 [==============================] - 55s 1ms/sample - loss: 0.0235 - accuracy: 0.9944 - val_loss: 0.0042 - val_accuracy: 0.9992
Epoch 2/500
49275/49275 [==============================] - 42s 847us/sample - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.0030 - val_accuracy: 0.9993
Epoch 3/

save 완료
